# Entrenamiento

## 1. Preprocesado

### 1.1. Lectura

In [22]:
import json
url, names, all_dictionaries = "./../generated/ml/models/", [ 'dictionary_manual_values.json' ], {}

for name in names:
    destiny = url + name.replace(' ', '%20')
    try:
        f = open(destiny, 'rb')
        all_dictionaries=json.loads(f.read())
    except OSError:
        print("No hay diccionario previo")
        all_dictionaries={}

In [23]:
all_dictionaries

{'make': {'ALFA ROMEO': 500,
  'AUDI': 1500,
  'MAZDA': 2500,
  'BMW': 3500,
  'CITROEN': 4500,
  'DACIA': 5500,
  'FIAT': 6500,
  'FORD': 7500,
  'HONDA': 8500,
  'HYUNDAI': 9500,
  'JAGUAR': 10500,
  'JEEP': 11500,
  'KIA': 12500,
  'LEXUS': 13500,
  'MERCEDES-BENZ': 14500,
  'MITSUBISHI': 15500,
  'MINI': 16500,
  'NISSAN': 17500,
  'PORSCHE': 18500,
  'OPEL': 19500,
  'SKODA': 20500,
  'PEUGEOT': 21500,
  'VOLVO': 22500,
  'RENAULT': 23500,
  'SEAT': 24500,
  'TOYOTA': 25500,
  'VOLKSWAGEN': 26500},
 'transmissionType': {'COMMON.TRANSMISSION_AUTOMATIC': 0,
  'COMMON.TRANSMISSION_MANUAL': 1},
 'seller_type': {'profesional': 0},
 'bodyType': {'BODY_TYPE_4X4': 500,
  'BODY_TYPE_SEDAN': 700,
  'BODY_TYPE_CABRIO': 900,
  'BODY_TYPE_FAMILIAR': 1100,
  'BODY_TYPE_COUPE': 1300,
  'BODY_TYPE_MINIVAN': 1500}}

In [24]:
import pandas as pd

url, names, dataframes = "./../generated/ml/preprocess/", [ 'all.csv' ], []

for name in names:
  destiny = url + name.replace(' ', '%20')
  print('Cargando: ' + destiny)
  if 'csv' in name:
    dataframes.append(pd.read_csv(destiny, delimiter=',', decimal=".", thousands=None))
  else:
    dataframes.append(pd.read_json(destiny))

Cargando: ./../generated/ml/preprocess/all.csv


In [25]:
all = pd.concat(dataframes)

### 1.2. Procesado de datos nulos

In [26]:
print("Datos iniciales | Longitud: {} | Num Columns: {}".format(len(all), len(all.columns)))
all=all.dropna(axis=1,how='all')
print("Eliminados na or null on columns | Longitud: {} | Num Columns: {}".format(len(all), len(all.columns)))
all=all.dropna(axis=0)
print("Eliminados na or null on rows | Longitud: {} | Num Columns: {}".format(len(all), len(all.columns)))
all=all.drop_duplicates()
all=all.drop_duplicates(subset=['id'], keep='first')
print("Eliminados duplicados | Longitud: {} | Num Columns: {}".format(len(all), len(all.columns)))
print("Datos finales | Longitud: {} | Num Columns: {}".format(len(all), len(all.columns)))

Datos iniciales | Longitud: 5086 | Num Columns: 36
Eliminados na or null on columns | Longitud: 5086 | Num Columns: 36
Eliminados na or null on rows | Longitud: 5086 | Num Columns: 36
Eliminados duplicados | Longitud: 5086 | Num Columns: 36
Datos finales | Longitud: 5086 | Num Columns: 36


### 1.3. Procesado columnas

In [27]:
all.columns

Index(['color', 'fuelTypeId', 'id', 'km', 'make', 'makeId', 'province',
       'provinceId', 'title', 'transmissionType', 'transmissionTypeId', 'year',
       'brand', 'brand_id', 'model', 'model_id', 'price', 'seller_type',
       'bodyType', 'cubicCapacity', 'doors', 'hp', 'acceleration',
       'consumptionAverage', 'consumptionExtraUrban', 'consumptionUrban',
       'emissions', 'fuelTankCapacity', 'height', 'length', 'luggageCapacity',
       'manufacturerPrice', 'seats', 'topSpeed', 'weight', 'width'],
      dtype='object')

In [28]:
all.columns

Index(['color', 'fuelTypeId', 'id', 'km', 'make', 'makeId', 'province',
       'provinceId', 'title', 'transmissionType', 'transmissionTypeId', 'year',
       'brand', 'brand_id', 'model', 'model_id', 'price', 'seller_type',
       'bodyType', 'cubicCapacity', 'doors', 'hp', 'acceleration',
       'consumptionAverage', 'consumptionExtraUrban', 'consumptionUrban',
       'emissions', 'fuelTankCapacity', 'height', 'length', 'luggageCapacity',
       'manufacturerPrice', 'seats', 'topSpeed', 'weight', 'width'],
      dtype='object')

In [29]:
columns_to_drop=['id','color','fuelTypeId','makeId','province','provinceId','title','transmissionTypeId','brand_id','luggageCapacity','consumptionAverage','consumptionExtraUrban','model_id','manufacturerPrice','seats','weight','emissions','fuelTankCapacity','height','luggageCapacity','brand','doors','topSpeed','consumptionUrban','model']
all=all.drop(columns_to_drop, axis = 1)

In [30]:
all.columns

Index(['km', 'make', 'transmissionType', 'year', 'price', 'seller_type',
       'bodyType', 'cubicCapacity', 'hp', 'acceleration', 'length', 'width'],
      dtype='object')

In [31]:
all.head(2)

,km,make,transmissionType,year,price,seller_type,bodyType,cubicCapacity,hp,acceleration,length,width
0,79990,ALFA ROMEO,COMMON.TRANSMISSION_AUTOMATIC,2018,29490,profesional,BODY_TYPE_4X4,2143.0,160.0,8.8,4687.0,1903.0
1,91718,ALFA ROMEO,COMMON.TRANSMISSION_AUTOMATIC,2018,30500,profesional,BODY_TYPE_4X4,2143.0,190.0,7.6,4687.0,1903.0


In [32]:
def columns_numeric_type(df,columns_names):
  for columns_name in columns_names:
    df[column_name]=pd.to_numeric(df[column_name])
  return df

In [33]:
qualitative_variable_names = [ e for e in all.select_dtypes(['object']).columns ]
for column_name in qualitative_variable_names:
  print(column_name, len(all[column_name].unique()))

make 27
transmissionType 2
seller_type 1
bodyType 6


In [34]:
import operator

def replace_for_numeric(df,column_name,initial,increaser,value_dictionary={}):
  values=df[column_name].unique()
  pos=initial
  if len(value_dictionary) > 0:
    max_dictionary=max(value_dictionary.items(), key=operator.itemgetter(1))[1]
  else:
    max_dictionary=initial
  if max_dictionary > pos:
    pos=max_dictionary+increaser
  for value in values:
    if value not in value_dictionary:
      value_dictionary[value]=pos
      df.loc[df[column_name] == value, column_name] = pos
      pos=pos+increaser
    else:
      df.loc[df[column_name] == value, column_name] = value_dictionary[value]
  df[column_name]=pd.to_numeric(df[column_name])
  # print("Nuevos valores: ", df[column_name].unique())
  return df,value_dictionary

In [35]:
all.select_dtypes(['object']).columns

Index(['make', 'transmissionType', 'seller_type', 'bodyType'], dtype='object')

In [36]:
all,word_dictionary=replace_for_numeric(all,'make',500,1000,all_dictionaries.get('make', {}))
all_dictionaries['make']=word_dictionary
all,word_dictionary=replace_for_numeric(all,'transmissionType',0,1,all_dictionaries.get('transmissionType', {}))
all_dictionaries['transmissionType']=word_dictionary
all,word_dictionary=replace_for_numeric(all,'seller_type',0,1,all_dictionaries.get('seller_type', {}))
all_dictionaries['seller_type']=word_dictionary
all,word_dictionary=replace_for_numeric(all,'bodyType',500,200,all_dictionaries.get('bodyType', {}))
all_dictionaries['bodyType']=word_dictionary
print(all_dictionaries)

{'make': {'ALFA ROMEO': 500, 'AUDI': 1500, 'MAZDA': 2500, 'BMW': 3500, 'CITROEN': 4500, 'DACIA': 5500, 'FIAT': 6500, 'FORD': 7500, 'HONDA': 8500, 'HYUNDAI': 9500, 'JAGUAR': 10500, 'JEEP': 11500, 'KIA': 12500, 'LEXUS': 13500, 'MERCEDES-BENZ': 14500, 'MITSUBISHI': 15500, 'MINI': 16500, 'NISSAN': 17500, 'PORSCHE': 18500, 'OPEL': 19500, 'SKODA': 20500, 'PEUGEOT': 21500, 'VOLVO': 22500, 'RENAULT': 23500, 'SEAT': 24500, 'TOYOTA': 25500, 'VOLKSWAGEN': 26500}, 'transmissionType': {'COMMON.TRANSMISSION_AUTOMATIC': 0, 'COMMON.TRANSMISSION_MANUAL': 1}, 'seller_type': {'profesional': 0}, 'bodyType': {'BODY_TYPE_4X4': 500, 'BODY_TYPE_SEDAN': 700, 'BODY_TYPE_CABRIO': 900, 'BODY_TYPE_FAMILIAR': 1100, 'BODY_TYPE_COUPE': 1300, 'BODY_TYPE_MINIVAN': 1500}}


### 1.4. Muestra de datos finales

In [37]:
all.columns

Index(['km', 'make', 'transmissionType', 'year', 'price', 'seller_type',
       'bodyType', 'cubicCapacity', 'hp', 'acceleration', 'length', 'width'],
      dtype='object')

In [38]:
all.tail(10)

,km,make,transmissionType,year,price,seller_type,bodyType,cubicCapacity,hp,acceleration,length,width
5076,23900,26500,0,2020,36700,0,700,1984.0,245.0,6.2,4258.0,1790.0
5077,129000,26500,0,2012,13900,0,700,1968.0,140.0,9.3,4199.0,1786.0
5078,86000,26500,1,2018,18900,0,700,1968.0,150.0,8.7,4767.0,1832.0
5079,55742,26500,1,2018,21399,0,500,999.0,115.0,10.1,4234.0,1819.0
5080,79235,26500,0,2017,29900,0,500,1968.0,150.0,9.3,4486.0,1839.0
5081,14149,26500,0,2020,25300,0,700,1498.0,150.0,8.3,4258.0,1790.0
5082,128428,26500,0,2018,23950,0,700,1968.0,150.0,8.7,4767.0,1832.0
5083,89000,26500,1,2016,27800,0,1500,1968.0,150.0,10.3,4854.0,1904.0
5084,64914,26500,0,2016,25990,0,700,1968.0,184.0,7.5,4268.0,1799.0
5085,66760,26500,0,2019,36099,0,500,1968.0,190.0,7.9,4486.0,1839.0
